<a href="https://colab.research.google.com/github/samarthkadaba/COVID-19-Cases-Predictor/blob/master/COVID_19_Neural_Network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from urllib.request import Request, urlopen
from bs4 import BeautifulSoup
import seaborn as sns
import re

from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense, Activation, Dropout
from tensorflow.python.keras.wrappers.scikit_learn import KerasRegressor
from tensorflow.python.keras.activations import relu, sigmoid, linear, exponential, softmax, tanh

In [0]:
def updateData():  

  ## Web-scraping data based on Worldometers.com COVID-19 daily updates

  site = "https://www.worldometers.info/coronavirus/country/us/"
  hdr = {'User-Agent': 'Mozilla/5.0'}
  req = Request(site,headers=hdr)
  pd.set_option('display.max_rows', None)
  pd.set_option('display.max_columns', None)
  page = urlopen(req)
  soup = BeautifulSoup(page)
  table = soup.find('table')
  table_rows = table.find_all('tr')
  col_labels = soup.find_all('th')
  all_headers = []
  col_str = str(col_labels)
  cleantext2 = BeautifulSoup(col_str, "lxml").get_text()
  all_headers.append(cleantext2)
  df2 = pd.DataFrame(all_headers)
  df3 = df2[0].str.split(',', expand=True)
  data = []
  for tr in table_rows:
      td = tr.find_all('td')
      row = [i.text for i in td]
      data.append(row)
  
  
  df = pd.DataFrame(data)
  frames = [df3, df]
  df4 = pd.concat(frames)
  df4[0] = df4[0].str.strip('[')
  df4.drop(df4.index[[0,2]], inplace = True)
  df4.drop(df4.columns[[2, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28]], axis=1, inplace=True)
  df4[0] = df4[0].str.replace(r'\n', '')
  df4[1] = (df4[1].str.replace(r',', '')).astype(float)
  df4[3] = df4[3].str.replace(r'\n', '')
  df4[3][51, 57, 58] = 0.00
  df4[3] = (df4[3].str.replace(r',', '')).astype(float)
  df4 = df4.fillna(0)


  df5 = df4.iloc[0:57, :]
  df5 = df5.reindex([5, 12, 8, 2, 9, 10, 17, 13, 20, 4, 3, 21, 11, 23, 7, 18, 16, 22, 19, 24, 15, 35, 25, 26, 6, 34, 32, 30, 14, 29, 37, 27, 38, 55, 28, 39, 42, 46, 31, 47, 45, 41, 44, 48, 36, 40, 49, 50, 52, 33, 43, 51, 53, 56, 57, 58])
  df5.drop(df5.index[[52, 53, 54, 55]], inplace = True)



 
    
  return df5


In [0]:

def loadData(df5):
  data = pd.read_csv('/content/COVID-19_Data_04062020.csv')
  data = pd.DataFrame(data, columns = ['example', 'state', 'pop', 'density', 'days', 'cases', 'deaths'])
  num_examples = data['example'].count()
  data = data.head(num_examples)
  numexamples = data.shape[0]
  dc = data.iloc[numexamples-52:numexamples, :]
  dc.loc[:, 'days'] = dc.loc[:, 'days']+1
  dc.loc[:, 'cases'] =  df5.iloc[:, 1].values
  dc.loc[:, 'deaths'] = df5.iloc[:, 2].values
  frames = [data, dc]
  data = pd.concat(frames)
  data.to_csv(r'/content/COVID-19_Data_04072020.csv', index = False)
  x = (data[['pop', 'density', 'days']])
  x = x.to_numpy()
  x = x.astype(float)
  y = (data[['cases']])
  y = y.to_numpy()
  y = y.astype(float)
  y = np.reshape(y, (-1,1))

  return x,y


In [0]:
def plotData(x, y, xlabel, ylabel):

  plt.xlabel(xlabel)
  plt.ylabel(ylabel)
  plt.scatter([x], [y], marker = 'o')

In [0]:
web_data = updateData()
x, y = loadData(web_data)

scaler_x = MinMaxScaler()
scaler_y = MinMaxScaler()
print(scaler_x.fit(x))
xscale=scaler_x.transform(x)
print(scaler_y.fit(y))
yscale=scaler_y.transform(y)

X_train, X_test, y_train, y_test = train_test_split(xscale, yscale)

In [27]:
def create_model(layers, activation):
  model = Sequential()
  for i, nodes in enumerate(layers):
   
    if(i == 0):
      model.add(Dense(nodes, input_dim = 3, kernel_initializer = 'normal', activation=activation))
      model.add(Dropout(0.25))
    else:
      model.add(Dense(nodes, activation = activation))
      

  model.add(Dense(1, activation = 'linear'))

  model.compile(loss = 'mse', optimizer = 'adam', metrics = ['mse', 'mae'])

  return model

model_checked = KerasRegressor(build_fn = create_model, verbose = 0)
model_checked

In [0]:
##Automatic Hyperparameter optimization with GridSearch CV

layers = [(64,32),  (64,32,16), (64)]
activations = ['sigmoid','exponential']
param_grid = dict(layers = layers, activation=activations, batch_size = [52,104,208], epochs = [500,1000,1500])
grid = GridSearchCV(estimator = model_checked, param_grid = param_grid, cv=KFold(n_splits = 3, shuffle=True))
grid_result = grid.fit(X_train, y_train)
print([grid_result.best_score_, grid_result.best_params_])

In [0]:
##Manually Update Network Paramters

model = Sequential()
model.add(Dense(64, input_dim = 3, kernel_initializer = 'normal', activation = 'sigmoid'))
model.add(Dropout(0.25))
model.add(Dense(32, activation = 'sigmoid'))
model.add(Dense(16, activation  = 'sigmoid'))
model.add(Dense(1, activation = 'linear'))
model.compile(loss = 'mse', optimizer = 'adam', metrics = ['mse', 'mae'])

In [36]:

##model = grid_result.best_estimator_
history = model.fit(X_train, y_train, verbose = 1, validation_split = 0.2, batch_size = 52, epochs = 1500)


Epoch 1/1500
5/5 [==============================] - 0s 23ms/step - loss: 0.0346 - mse: 0.0352 - mae: 0.1589 - val_loss: 0.0094 - val_mse: 0.0137 - val_mae: 0.0662
Epoch 2/1500
5/5 [==============================] - 0s 7ms/step - loss: 0.0220 - mse: 0.0203 - mae: 0.0619 - val_loss: 0.0115 - val_mse: 0.0177 - val_mae: 0.0688
Epoch 3/1500
5/5 [==============================] - 0s 7ms/step - loss: 0.0228 - mse: 0.0228 - mae: 0.0725 - val_loss: 0.0089 - val_mse: 0.0144 - val_mae: 0.0383
Epoch 4/1500
5/5 [==============================] - 0s 7ms/step - loss: 0.0181 - mse: 0.0178 - mae: 0.0516 - val_loss: 0.0090 - val_mse: 0.0134 - val_mae: 0.0610
Epoch 5/1500
5/5 [==============================] - 0s 9ms/step - loss: 0.0183 - mse: 0.0182 - mae: 0.0760 - val_loss: 0.0119 - val_mse: 0.0156 - val_mae: 0.0881
Epoch 6/1500
5/5 [==============================] - 0s 86ms/step - loss: 0.0193 - mse: 0.0190 - mae: 0.0830 - val_loss: 0.0095 - val_mse: 0.0137 - val_mae: 0.0676
Epoch 7/1500
5/5 [========

In [0]:
print(history.history.keys())
#Loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc = 'upper left')
plt.show()

In [0]:
x, y = loadData(web_data)
plotData(x[:, 0], y, 'Population', 'Number of Cases')
plt.figure()
plt.axis([0, 2000, 0, 15000])
plotData(x[:, 1], y, 'Population Denisty', 'Number of Cases')
plt.figure()
plt.axis([0, 70, 0, 10000])
plotData(x[:, 2], y, 'Days Since First Case', 'Number of Cases')

In [0]:
## Compare prediction on entire training set to confirm linearity 

x, y = loadData(web_data)
x = np.expand_dims(x, axis = 1)
predictions = []
for i in range(len(x)):
  x[i]= scaler_x.transform(x[i].reshape(1,-1))
  ynew= model.predict(x[i])
  #invert normalize
  ynew = scaler_y.inverse_transform(ynew.reshape(1,-1))
  x[i] = scaler_x.inverse_transform(x[i])
  predictions.append(ynew[0])
##print("X=%s, Predicted=%s" % (Xnew[0], ynew[0]))
plt.figure()
plt.axis([0, 10000, 0, 10000])
plotData(y, predictions, 'measured', 'predicted' )
for i in range(len(predictions)):
  print(i+1, predictions[i])


In [0]:
## Extrapolation predictions

pop = float(input("Enter the popualtion: "))
dense = float(input("Enter the population density: "))
days = float(input("Enter the days since first case: "))
x_spec = [[pop, dense, days]]
x_spec = scaler_x.transform(x_spec)
y_hat = model.predict(x_spec)
y_hat = scaler_y.inverse_transform(y_hat.reshape(-1,1))
x_spec = scaler_x.inverse_transform(x_spec)
print("X: %s, Predicted = %s" % (x_spec[0], y_hat[0]))